<a href="https://colab.research.google.com/github/MicaelaPalaciosD/data-science/blob/main/econometrics3assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


# sort data per year
data_original = pd.read_csv("data_assignment1.csv")
data_sorted_years = [data_original.iloc[i::7].reset_index(drop=True) for i in range(7)]
N=595
T=7


In [ ]:
# regression per year
beta_i = []
standard_error_i=[]
t_stat_i=[]

for i in range(7):
    data = data_sorted_years[i].dropna()
    y = data["lwage"].to_numpy()
    X = data.iloc[:, 2:-1].to_numpy()
    X = np.column_stack((np.ones(len(X)), X))

    # LS
    beta = np.linalg.lstsq(X, y, rcond=None)[0]
    beta_i.append(beta)

    #Residual
    y_hat = X @ beta
    residuals = y - y_hat

    n = len(y)
    k = X.shape[1]
    sigma2 = (residuals.T @ residuals) / (n - k)

    # Variance-covariance matrix
    var_beta = sigma2 * np.linalg.inv(X.T @ X)

    #Standard error/t-stat
    standard_error = np.sqrt(np.diag(var_beta))
    standard_error_i.append(standard_error)

    t_stat = beta / standard_error
    t_stat_i.append(t_stat)

for i in range(7):
  print("Beta 0 for year", i, "=", beta_i[i][0])
  print("Beta 1 for year", i, "=", beta_i[i][1])
  print("Standard error for year", i, "=", standard_error_i[i])
  print("t_statistic for year", i, "=", t_stat_i[i])


Beta 0 for year 0 = 5.203449982235075
Beta 1 for year 0 = 0.00985229887779484
Standard error for year 0 = [0.13386525 0.00113273 0.00187673 0.0305063  0.02502842 0.02648924
 0.0258515  0.04582977 0.0549637  0.02692124 0.0055242  0.04672296]
t_statistic for year 0 = [38.8708057   8.6978598   3.12842311 -4.14005339  0.79746745 -2.10838794
  6.98976847  2.05896064 -5.28317281  4.49194466  8.60540275 -2.96340775]
Beta 0 for year 1 = 5.610636143856355
Beta 1 for year 1 = 0.007504823748708152
Standard error for year 1 = [0.13802042 0.00105679 0.00217002 0.02863984 0.02329041 0.02483715
 0.02436928 0.04165552 0.05061849 0.02565947 0.00515679 0.04364471]
t_statistic for year 1 = [40.65076931  7.10153454  0.31329502 -3.66806105  0.75466151 -2.37848812
  6.40503746  1.69439008 -6.82677282  4.24515826  9.09654381 -3.12716686]
Beta 0 for year 2 = 5.707219438305201
Beta 1 for year 2 = 0.007902889582629202
Standard error for year 2 = [0.18913659 0.00137615 0.00304165 0.03772125 0.03060433 0.03255334

In [ ]:
# pooled regression
y_pooled = data_original["lwage"].to_numpy()
X_pooled = data_original.iloc[:, 2:-1].to_numpy()
X_pooled = np.column_stack((np.ones(len(X_pooled)), X_pooled))

  # LS
beta_pooled = np.linalg.lstsq(X_pooled, y_pooled, rcond=None)[0]

#residuals
y_hat_pooled = X_pooled @ beta_pooled
residuals_pooled = y_pooled - y_hat_pooled

n_pooled = len(y_pooled)
k_pooled = X_pooled.shape[1]
sigma2_pooled = (residuals_pooled.T @ residuals_pooled) / (n_pooled - k_pooled)

# Variance-covariance matrix
var_beta_pooled = sigma2_pooled * np.linalg.inv(X_pooled.T @ X_pooled)

#Standard error/t-stat
standard_error_pooled = np.sqrt(np.diag(var_beta_pooled))
t_stat_pooled = beta_pooled / standard_error_pooled

print("Beta 0 pooled =", beta_pooled[0])
print("Beta 1 pooled =", beta_pooled[1])
print("Standard error pooled=", standard_error_pooled)
print("t_statistic pooled=", t_stat_pooled)



Beta 0 pooled = 5.441157911177231
Beta 1 pooled = 0.010370901395587852
Standard error pooled= [0.07168802 0.00053535 0.00110591 0.01499334 0.01206629 0.01282467
 0.012348   0.02102077 0.02567434 0.01308703 0.00267494 0.02256787]
t_statistic pooled= [ 75.90052266  19.37216353   4.47096255  -9.91335509   4.39719069
  -4.14904103  11.76740201   3.14355239 -13.76088272   7.79976828
  21.36641919  -7.40533187]


Question A


In [ ]:
def between_estimation(data):
  # create person id (each person has 7 years of observations)
  T = 7
  data["person"] = (np.arange(data.shape[0]) // T) + 1

  #define dependent variable and regressors
  y = "lwage"
  X = ["exp", "wks", "bluecol", "ind", "south", "smsa",
          "married", "gender", "union", "edu", "col"]

  #compute individual time-averages (between transformation)
  means = data.groupby("person")[[y] + X].mean()
  Yb = means[y].to_numpy().reshape(-1, 1)
  Xb = means[X].to_numpy()

    # add constant and set dimensions
  N = Xb.shape[0]
  k = Xb.shape[1]
  Xb = np.hstack((np.ones((N, 1)), Xb))

  # estimate between coefficients using OLS matrix formula


  XtX_inverse = np.linalg.inv(Xb.T @ Xb)
  beta_hat_b = XtX_inverse @(Xb.T @ Yb)

  a_b = Yb - Xb @ beta_hat_b

  # compute between residual variance (sigma^2_alpha)
  df_b = N - k
  s2_b = ((a_b.T @ a_b) / df_b).item()

  # compute standard errors and t-statistics
  Vb = s2_b * np.linalg.inv(Xb.T @ Xb)
  se_b = np.sqrt(np.diag(Vb)).reshape(-1, 1)
  t_b = beta_hat_b / se_b

    # compute R-squared for the between regression
  Ybar = np.mean(Yb)
  TSS = np.sum((Yb - Ybar) ** 2)
  SSR = np.sum(a_b ** 2)
  R2_b = 1 - SSR / TSS


  # print final results
  names = ["const"] + X

  print("\n===== BETWEEN ESTIMATOR RESULTS =====")
  print("\nR2 =", R2_b)
  print("\nsigma 2 alpha =", s2_b)
  print("\nvariance", Vb)

  for j in range(len(names)):
        print(names[j],
            "coef =", beta_hat_b[j].item(),
            "se =", se_b[j].item(),
            "t =", t_b[j].item())

  return beta_hat_b, Vb, s2_b



In [ ]:
data = pd.read_csv("data_assignment1.csv")
b_beta, b_var, s2_alpha = between_estimation(data)


===== BETWEEN ESTIMATOR RESULTS =====

R2 = 0.5214981702236977

sigma 2 alpha = 0.0756442268392153

variance [[ 4.29294390e-02 -5.35694532e-05 -6.61826617e-04 -1.82348924e-03
  -5.40564284e-05 -5.95650629e-04  1.32870121e-04 -1.53173413e-03
  -3.07021353e-03 -1.85835279e-03 -5.77791128e-04 -3.67916382e-04]
 [-5.35694532e-05  1.25374469e-06  3.99080995e-07  2.85155208e-06
  -3.14083864e-06  2.06532888e-06 -3.32750614e-06 -9.37745442e-06
  -3.03233448e-06  2.03560571e-06  1.46397192e-06 -2.92052407e-06]
 [-6.61826617e-04  3.99080995e-07  1.35621829e-05 -5.07724691e-06
  -6.31980027e-06  2.55401332e-07 -9.85731879e-06 -6.24717732e-06
   1.86806040e-05  3.43436976e-05  1.61976665e-06  2.47363847e-06]
 [-1.82348924e-03  2.85155208e-06 -5.07724691e-06  1.19425944e-03
  -8.55529169e-05 -8.60627511e-06  1.40312487e-04  1.65323469e-05
   9.42915801e-05 -3.09536911e-04  1.11587576e-04 -8.46584120e-05]
 [-5.40564284e-05 -3.14083864e-06 -6.31980027e-06 -8.55529169e-05
   6.81473382e-04  6.1042258

Question B

In [ ]:


def within_transformation(data, y_var, x_var, indiv):
    #####substracting individual mean

    y_mean = data.groupby(indiv)[y_var].transform("mean")
    x_mean = data.groupby(indiv)[x_var].transform("mean")

    y_tilde = np.array(data[y_var] - y_mean)
    x_tilde = np.array(data[x_var] - x_mean)

    return y_tilde, x_tilde


def within_estimator(y_tilde, x_tilde, N, T):

    ### dimenties, NT is het aantal observations, k regressors

    NT, k= x_tilde.shape

    ###betahat
    XtX_inverse = np.linalg.inv(x_tilde.T @ x_tilde)
    beta_hat = XtX_inverse @(x_tilde.T @ y_tilde)

    residuals = y_tilde - x_tilde @ beta_hat

    ###standard error
    sigma2_eta = (residuals.T @ residuals)/(N*(T-1)-k)


    var_beta_hat = sigma2_eta * XtX_inverse ####variance-covariance matrix
    se_beta_hat= np.sqrt(np.diag(var_beta_hat))

    t_values = beta_hat.flatten()/ se_beta_hat

    SSR= residuals.T @ residuals
    SST = y_tilde.T @ y_tilde
    R2 = 1- (SSR/SST)

    return beta_hat, se_beta_hat, t_values, R2, sigma2_eta, var_beta_hat

#### print result

data = pd.read_csv("data_assignment1.csv")
data["indiv"] = np.repeat(np.arange(595), 7)


    ## leave out variable that does not change
time_varying_var = ["exp", "wks", "bluecol", "ind","south","married", "union"]

y_within, x_within = within_transformation(data, "lwage", time_varying_var, "indiv")
beta_within, se_within, t_within, R2_within, sigma2_within, var_within = within_estimator(y_within, x_within, 595,7)

results_table = pd.DataFrame({
  "Variable": time_varying_var,
  "Beta": beta_within.flatten(),
  "Std. Error": se_within,
  "t-value": t_within
    })

# Rounding for cleaner output
results_table = results_table.round(4)

print("\n===== WITHIN ESTIMATOR RESULTS =====")
print(results_table.to_string(index=False))

print("\nWithin R2 =", round(R2_within,4))
print("Sigma^2_eta =", round(sigma2_within,6))


===== WITHIN ESTIMATOR RESULTS =====
Variable    Beta  Std. Error  t-value
     exp  0.0967      0.0012  81.2735
     wks  0.0011      0.0006   1.8865
 bluecol -0.0250      0.0139  -1.7967
     ind  0.0202      0.0156   1.2963
   south -0.0109      0.0344  -0.3155
 married -0.0331      0.0191  -1.7343
   union  0.0336      0.0150   2.2343

Within R2 = 0.652
Sigma^2_eta = 0.023503


**Question c**

In [ ]:
def fgls(data, y_var, x_var, sigma2_eta, sigma2_alpha, N, T):
    y = data[y_var].to_numpy()
    X = data[x_var].to_numpy()
    X = np.column_stack((np.ones(len(X)), X))
    k = X.shape[1]

# theta sqrd

    theta2 = sigma2_eta / (T * sigma2_alpha + sigma2_eta)
    theta = np.sqrt(theta2)

#getting X* and Y*
    sigma_eta = np.sqrt(sigma2_eta)
    X_star = []
    y_star = []

    for i in range (N):
        Xi = X[i * T : (i + 1) * T, :]
        yi = y[i * T : (i + 1) * T]

        Xi_bar= Xi.mean(axis=0)
        yi_bar= yi.mean()

        Xi_star = (Xi - (1 - theta) * Xi_bar) / sigma_eta
        yi_star = (yi - (1 - theta) * yi_bar) / sigma_eta
        X_star.append(Xi_star)
        y_star.append(yi_star)

    X_star = np.vstack (X_star)
    y_star = np.concatenate (y_star)

# calculating GLS
    XtX_inverse = np.linalg.inv(X_star.T @ X_star)
    beta_FGLS = XtX_inverse @(X_star.T @ y_star)

#calculating Var
    var_beta_FGLS= XtX_inverse

    #calculating SE
    se_FGLS= np.sqrt(np.diag(var_beta_FGLS))
    t_stat_FGLS = beta_FGLS/se_FGLS

    # Calculating Rsqrd
    y_hat_star = X_star @ beta_FGLS
    residuals_star = y_star - y_hat_star

    SSR_star = residuals_star.T @ residuals_star
    SST_star = (y_star - y_star.mean()).T @ (y_star - y_star.mean())

    R2 = 1 - SSR_star / SST_star

    print(X_star)
    print("Estimated beta using FGLS", beta_FGLS)

    print("Estimated Standard error using FGLS", se_FGLS)

    print("Estimated t-statistic using FGLS", t_stat_FGLS)

    print("Estimated R-squared using FGLS", R2)

    return beta_FGLS, var_beta_FGLS

Question d


In [ ]:
def hausman_test (beta_within, beta_FGLS, var_within, Sigma_FGLS):
    beta_diff = beta_within .flatten() - beta_FGLS.flatten()

    var_diff= var_within - Sigma_FGLS

    hausman= beta_diff.T @ np.linalg.inv(var_diff)@ beta_diff

    return hausman

Question D


Question e

In [ ]:

# order data

y = data_original["lwage"].to_numpy()
X = data_original.iloc[:, 2:-1].to_numpy()
k = X.shape[1]

y_ordered = y.reshape(N, T).T
X_ordered = X.reshape(N, T, k).transpose(1, 0, 2)

# transform data
y_i_star= y_ordered.mean(axis=0)
X_i_star= X_ordered.mean(axis=0)

y_star_t= y_ordered.mean(axis=1)
X_star_t= X_ordered.mean(axis=1)

y_star_star=y_ordered.mean()
X_star_star=X_ordered.mean(axis=(0,1))

#demean to remove individual and time effect
y_hat_two_way=y_ordered-y_i_star-y_star_t[:, None]+y_star_star
X_hat_two_way=X_ordered-X_i_star-X_star_t[:, None, :]+X_star_star

#Reshape to fit matrices
y_hat_two_way = y_hat_two_way.reshape(T*N, 1)
X_hat_two_way = X_hat_two_way.reshape(T*N, k)


#check for collinearity and remove variables that are 0
col_std = X_hat_two_way.std(axis=0)
non_zero_regressors = col_std > 0
X_hat_two_way = X_hat_two_way[:, non_zero_regressors]

# Print remaining column names
X_df = data_original.iloc[:, 2:-1]
X_df = X_df.loc[:, non_zero_regressors]

print(X_df.iloc[0])


#perform OLS
beta_hat_two_way = np.linalg.inv(X_hat_two_way.T @ X_hat_two_way) @ (X_hat_two_way.T @ y_hat_two_way)

#calculate Sigma_squared
residuals=(y_hat_two_way-X_hat_two_way@beta_hat_two_way)
degrees_of_freedom= (T*N - X_hat_two_way.shape[1])
sigma_squared_two_way = (residuals.T @ residuals).item() / degrees_of_freedom


# variance–covariance matrix
var_covar_beta = sigma_squared_two_way * np.linalg.inv(X_hat_two_way.T @ X_hat_two_way)

# standard errors (one per regressor)
SE_two_way = np.sqrt(np.diag(var_covar_beta))

#calculate t-stat
t_stats_two_way = beta_hat_two_way.flatten()/ SE_two_way

#calculate R-squared
SSR_two_way = (residuals.T @ residuals).item()
SST_two_way = (y_hat_two_way.T @ y_hat_two_way).item()
R_squared_two_way= 1- (SSR_two_way/SST_two_way)

print("Estimated beta using two way fixed effects", beta_hat_two_way)

print("Estimated Standard error using two way fixed effects", SE_two_way)

print("Estimated t-statistic using two way fixed effects", t_stats_two_way)

print("Estimated R-squared using two way fixed effects", R_squared_two_way)


wks        32
bluecol     0
ind         0
south       1
smsa        0
married     1
union       0
Name: 0, dtype: int64
Estimated beta using two way fixed effects [[ 0.00094862]
 [-0.02213145]
 [ 0.02235811]
 [ 0.00228936]
 [-0.04318164]
 [-0.02899049]
 [ 0.03067453]]
Estimated Standard error using two way fixed effects [0.00055712 0.01280395 0.01434635 0.03185325 0.01805003 0.01762711
 0.01386421]
Estimated t-statistic using two way fixed effects [ 1.70270601 -1.72848667  1.55845277  0.0718722  -2.39233087 -1.6446541
  2.21249842]
Estimated R-squared using two way fixed effects 0.005131707838780142


In [ ]:

def main():


    N= 595
    T= 7

    data["indiv"] = np.repeat(np.arange(N), T)


    ## leave out variable that does not change
    time_varying_var = ["exp", "wks", "bluecol", "ind","south", "smsa", "married", "union"]


    ##### within##########
    y_within, x_within = within_transformation(data, "lwage", time_varying_var, "indiv")
    beta_within, se_within, t_within, R2_within, sigma2_within, var_within = within_estimator(y_within, x_within, N,T)

    results_within_table = pd.DataFrame({
    "Variable": time_varying_var,
    "Beta": beta_within.flatten(),
    "Std. Error": se_within,
    "t-value": t_within
    })

# Rounding for cleaner output
    results_within_table = results_within_table.round(4)

    print("\n===== WITHIN ESTIMATOR RESULTS =====")
    print(results_within_table.to_string(index=False))

    print("\nWithin R2 =", round(R2_within,6))
    print("\nSigma^2_eta =", round(sigma2_within,6))

    # between#######################################################

    beta_between, var_between, sigma2_alpha = between_estimation(data)

 # FGLS#######################################################
    print("\n===== FGLS RESULTS =====")

    x_var= ["exp", "wks", "bluecol", "ind", "south", "smsa",
          "married", "gender", "union", "edu", "col"]

    beta_fgls, var_fgls=fgls(data, "lwage", x_var, sigma2_within, sigma2_alpha, N, T)

    # FGLS ( same regressors as within)
    print("\n===== FGLS adjust RESULTS =====")
    beta_adj_fgls, var_adj_fgls = fgls(data, "lwage", time_varying_var,
                       sigma2_within, sigma2_alpha,
                       N, T)


######hausman

   #remove constant from between
    beta_re= beta_adj_fgls[1:]  #drop intercept
    var_re= var_adj_fgls[1:, 1:]   #drop first row& column

    H = hausman_test(beta_within, beta_re, var_within, var_re)

    print("\n===== HAUSMAN TEST =====")
    print("H-statistic =", round(H,4))


###########################################################
### call main
if __name__ == "__main__":
    main()



===== WITHIN ESTIMATOR RESULTS =====
Variable    Beta  Std. Error  t-value
     exp  0.0966      0.0012  81.0992
     wks  0.0011      0.0006   1.8937
 bluecol -0.0249      0.0139  -1.7904
     ind  0.0208      0.0156   1.3331
   south -0.0032      0.0346  -0.0925
    smsa -0.0437      0.0196  -2.2327
 married -0.0303      0.0191  -1.5812
   union  0.0342      0.0150   2.2708

Within R2 = 0.65251

Sigma^2_eta = 0.023477

===== BETWEEN ESTIMATOR RESULTS =====

R2 = 0.5214981702236977

sigma 2 alpha = 0.0756442268392153

variance [[ 4.29294390e-02 -5.35694532e-05 -6.61826617e-04 -1.82348924e-03
  -5.40564284e-05 -5.95650629e-04  1.32870121e-04 -1.53173413e-03
  -3.07021353e-03 -1.85835279e-03 -5.77791128e-04 -3.67916382e-04]
 [-5.35694532e-05  1.25374469e-06  3.99080995e-07  2.85155208e-06
  -3.14083864e-06  2.06532888e-06 -3.32750614e-06 -9.37745442e-06
  -3.03233448e-06  2.03560571e-06  1.46397192e-06 -2.92052407e-06]
 [-6.61826617e-04  3.99080995e-07  1.35621829e-05 -5.07724691e-06
 